In [1]:
# import os, glob

import geopandas as gpd
import pandas as pd
# import rasterio as rs
# import rioxarray as rxr
# import numpy as np
# import boto3

# from rasterio.session import AWSSession
# from rasterio.enums import Resampling
from pystac_client import Client
from shapely.geometry import shape, box

# import matplotlib.pyplot as plt
# %matplotlib notebook

# Get search parameters

In [4]:
# boundaries of thwaites eastern shelf
aoi_bounds = -1609000,-465000,-1542000,-415000

# get date range
date1 = '2019-01-01'
date2 = '2019-03-01'


In [5]:
# Process to search inputs
aoi_geom = box(*aoi_bounds)
aoi_gdf = gpd.GeoDataFrame(geometry=[aoi_geom], crs=3031)
aoi_geom_4326 = aoi_gdf.to_crs(4326).geometry.values[0]

daterange= f'{date1}/{date2}'

# Search Landsat AWS STAC

In [4]:
# Open client instance
landsat_stac_url = 'https://landsatlook.usgs.gov/stac-server'
landsatlookstac = Client.open(landsat_stac_url, headers=[])


In [5]:
# Assess and select a collection
[i for i in landsatlookstac.get_collections()]

[<CollectionClient id=landsat-c2l2-sr>,
 <CollectionClient id=landsat-c2l2-st>,
 <CollectionClient id=landsat-c2ard-st>,
 <CollectionClient id=landsat-c2l2alb-bt>,
 <CollectionClient id=landsat-c2l3-fsca>,
 <CollectionClient id=landsat-c2ard-bt>,
 <CollectionClient id=landsat-c2l1>,
 <CollectionClient id=landsat-c2l3-ba>,
 <CollectionClient id=landsat-c2l2alb-st>,
 <CollectionClient id=landsat-c2ard-sr>,
 <CollectionClient id=landsat-c2l2alb-sr>,
 <CollectionClient id=landsat-c2l2alb-ta>,
 <CollectionClient id=landsat-c2l3-dswe>,
 <CollectionClient id=landsat-c2ard-ta>]

In [6]:
search = landsatlookstac.search(
    intersects=aoi_geom_4326,
    datetime=daterange,
    collections=["landsat-c2l1"],
    limit=500,      # number of items per page of results
    max_items=1e9,  # abritrarily large max_items to return.
)

items = [i.to_dict() for i in search.get_items()]
items = pd.json_normalize(items)

In [8]:
# # Print pandas dataframe of search results
# items

# Search PGC AWS Stac

In [2]:
# pgc_stac_url = 'https://polargeospatialcenter.github.io/stac-browser'
# pgc_stac_url = 'https://pgc-opendata-dems.s3.us-west-2.amazonaws.com/rema/mosaics.json'
pgc_stac_url = 'https://pgc-opendata-dems.s3.us-west-2.amazonaws.com/pgc-data-stac.json'

pgcstac = Client.open(pgc_stac_url, headers=[])


In [3]:
[i for i in pgcstac.get_collections()]

[<CollectionClient id=arcticdem>,
 <CollectionClient id=earthdem>,
 <CollectionClient id=rema>]

In [6]:
pgcstac.search(
    intersects=aoi_geom_4326,
    datetime=daterange,
    collections=["rema"],
    limit=500,      # number of items per page of results
    max_items=1e9,  # abritrarily large max_items to return.
)

NotImplementedError: This catalog does not support search because it does not conform to "ConformanceClasses.ITEM_SEARCH"